# Enunciado


2. Usando o protocolo  OT construído na questão anterior
    1. Implemente o protocolo $\,\mathsf{sVOLE}\;$ (“subset vectorial oblivious linear evaluation”)
    2. Usando $\,\mathsf{sVOLE}\,$   implemente um protótipo de um protocolo  ZK-sVOLE usando equações polinomiais do 2º grau aleatoriamente geradas.

----------

# PARTE A

## Definição de variáveis

Para definir o protocolo vamos começar por fixar os parâmetros, $\,p\,$, $\,k\,$ e $\,\ell\,$, que determinam a estrutura algébrica onde o protocolo se desenvolve.

In [1]:
p = 3
k = 5
l = 8
F = GF(p)
R.<x> = PolynomialRing(F)
lambda_security = 128
f = R.irreducible_element(k)
E.<a> = GF(p^k, name='a', modulus=f)
E

Finite Field in a of size 3^5

Esta estrutura algébrica fornece a base para o protocolo OLE, que permite a avaliação linear de forma “oblivious”, sendo uma primitiva fundamental em construção de protocolos criptográficos como OT.

## Oblivious transfer da questão anterior:

In [2]:
import random
import hashlib

def bernoulli(epsilon, n=53):
    """
    Gera uma amostra de Bernoulli B(epsilon) usando a construção de Lebesgue.
    - epsilon: parâmetro da distribuição de Bernoulli (0 < epsilon < 1)
    - n: número de bits para a precisão (default: 53, precisão de um double)
    """
    # Gera a string de bits aleatórios {0,1}^n
    w = [random.randint(0, 1) for _ in range(1, n+1)]
    
    # Calcula o racional de Lebesgue
    w_hat = sum(w[i-1] * 2^(-i) for i in range(1,n+1))
    
    return 1 if w_hat <= epsilon else 0

def bernoulli_lambda(epsilon, n=53, lambda_ = lambda_security):
    return [bernoulli(epsilon,n) for _ in range(lambda_)]

def vector_to_bytes(vector, p):
    """Converte um vetor F_p^ℓ em bytes."""
    byte_array = bytearray()
    for elemento in vector:
        # Assume que elemento está em {0, ..., p-1}
        byte_array.append(elemento % p)  # 1 byte por elemento (se p ≤ 256)
    return bytes(byte_array)

def bytes_to_vector(byte_string, p=257, l=10):
    """Converte bytes de volta para F_p^ℓ."""
    vector = []
    for byte in byte_string[:l]:
        vector.append(byte % p)
    return vector

def xof(seed: bytes, nbytes: int):
    shake = hashlib.shake_128()
    shake.update(seed)
    return shake.digest(nbytes)  # retorna nbytes de saída
    
def xof_bits(seed: bytes, nbits: int):
    nbytes = (nbits + 7) // 8
    output = xof(seed, nbytes)
    bits = []
    for byte in output:
        for i in range(8):
            bits.append((byte >> (7 - i)) & 1)
            if len(bits) == nbits:
                return bits
                
def bytes_to_bits(byte_string):
    bits = []
    for byte in byte_string:
        for i in range(8):
            bits.append((byte >> i) & 1)
    return bits

def bits_to_bytes(bits):
    byte_array = bytearray()
    for i in range(0, len(bits), 8):
        byte = 0
        for j in range(8):
            if i + j < len(bits):
                byte |= bits[i + j] << j
        byte_array.append(byte)
    return bytes(byte_array)

In [3]:
class Sender:
    def __init__(self, alpha, l, xof_name="shake128"):
        self.alpha = alpha
        self.l = l
        self.xof_name = xof_name
        self.seed = f"{alpha}:{l}".encode()  # usa α e ℓ como seed
        self.criterion = None
        self.pks = []
        self.N = p^k        
        self.mensagens = self.gerar_mensagens()
        self.r = []
        self.criptogramas = []
        self.t = p^k #Número de criptogramas a serem feitos para enviar ao Receiver
    
    def gerar_mensagens(self):
        n_bytes = 6
        return [f"msg{i:03d}".encode().ljust(n_bytes, b'\x00')[:n_bytes] for i in range(self.N)]

    def get_vector_ai_ui(self, i, lambda_):
        """Retorna (a_i, u_i) ∈ F₂^λ × F₂, para índice i"""
        # gera (λ + 1) bits para cada i
        seed_i = self.seed + f":{i}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]  # bit extra
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        """Gera todos os pares (a_i, u_i) com base na XOF"""
        self.criterion = [self.get_vector_ai_ui(i, lambda_) for i in range(1, self.l + 1)]

    def receive_and_verifypks(self, pks):
        self.pks = pks
        for i in range(self.l):
            soma = sum(pks[i][k] for k in range(self.N)) % 2
            _, u_i = self.criterion[i]  # a_i não é necessário aqui
            if soma != u_i:
                print(f"FALHA na linha i={i}: soma={soma}, u_i={u_i}")
                return False
        print("Verificação OK: todas as somas coincidem com u_i")
        return True

    def gerar_r(self, delta, p=0.1):
        while True:
            r = [bernoulli(p) for _ in range(self.l)]
            if sum(r) <= delta:
                self.r = r
                return

    def calcular_criptogramas(self):
        for _ in range(self.t):
            self.gerar_r(128)
            a = [0] * lambda_security
            for i in range(self.l):
                if self.r[i] == 1:
                    a_i, _ = self.criterion[i]
                    a = [(a[j] + a_i[j]) % 2 for j in range(lambda_security)]
            
            criptogramas_k = []
            for k in range(self.N):
                # Converte o elemento de F para seu valor inteiro (0, 1 ou 2)
                valor = int(self.mensagens[k])
                # Converte o valor inteiro para bits (2 bits suficientes para valores 0-2)
                msg_bits = [(valor >> i) & 1 for i in range(2)]  # Usamos 2 bits
                c_k = []
                for bit in msg_bits:
                    soma = sum(self.r[i] * self.pks[i][k] for i in range(self.l)) % 2
                    c_k.append((bit + soma) % 2)
                criptogramas_k.append(c_k)
            self.criptogramas.append((a, criptogramas_k))
            
    def print_info(self):
        print("self.alpha: ",self.alpha)
        print("self.l: ",self.l)
        print("self.xof_name: ",self.xof_name)
        print("self.seed: ",self.seed)
        print("self.criterion: ",self.criterion)

In [4]:
class Receiver:
    def __init__(self, eps, b, lambda_, n_mensagens):
        self.alpha = None
        self.l = None
        self.eps = eps
        self.secrets = []
        self.criterion = None
        self.N = n_mensagens
        self.t = None
        self.b = b 

    def receive_alpha_l(self, alpha, l):
        self.alpha = alpha
        self.l = l
        self.seed = f"{alpha}:{l}".encode()
        self.criterion = self.get_criterion_sequence(lambda_security)

    def get_vector_ai_ui(self, i, lambda_):
        seed_i = self.seed + f":{i}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        return [self.get_vector_ai_ui(i, lambda_) for i in range(1, self.l + 1)]

    def generate_N_secrets(self):
        self.secrets = []
        for k in range(self.N):
            if k == self.b:
                self.secrets.append(None)  # s_b = ⊥
            else:
                s_k = bernoulli_lambda(self.eps, lambda_security)
                self.secrets.append(s_k)

    def generate_pks(self):
        # Inicializar t[i][k]
        self.t = [[0 for _ in range(self.N)] for _ in range(self.l)]  
        for i in range(self.l):
            a_i, u_i = self.criterion[i]
            soma = 0
            for k in range(self.N):
                if k != self.b:
                    s_k = self.secrets[k] 
                    dot = sum([a_i[j] * s_k[j] for j in range(lambda_security)]) % 2 
                    e = bernoulli(self.eps)
                    t_ik = (dot + e) % 2
                    self.t[i][k] = t_ik
                    soma = (soma + t_ik) % 2
                else:
                    self.t[i][k] = None 
            
            # Calcula t_{i,b} como complemento para somar a u_i
            self.t[i][self.b] = (u_i - soma) % 2

    def recuperar_mensagens(self, a, c):
        mensagens_recuperadas = []
        for k in range(self.N):
            if k == self.b:
                mensagens_recuperadas.append(None)
            else:
                s_k = self.secrets[k]
                a_dot_s = sum(a[j] * s_k[j] for j in range(lambda_security))
                bits_recuperados = [(c[k][i] + a_dot_s) % 2 for i in range(len(c[k]))]
                msg_bytes = bits_to_bytes(bits_recuperados)
                mensagens_recuperadas.append(msg_bytes)
        return mensagens_recuperadas

    def recuperar_mensagens_maioritarias(self, lista_criptogramas):
        t = len(lista_criptogramas)
        resultados_por_k = [[] for _ in range(self.N)]
        
        for i in range(t):
            a, c = lista_criptogramas[i]
            mks = self.recuperar_mensagens(a, c)
            for k in range(self.N):
                resultados_por_k[k].append(mks[k])
        
        mensagens_finais = []
        for k in range(self.N):
            if k == self.b:
                mensagens_finais.append(None)
            else:
                # Encontra a mensagem mais frequente (votação majoritária)
                contagem = {}
                for msg in resultados_por_k[k]:
                    contagem[msg] = contagem.get(msg, 0) + 1
                mensagem_mais_votada = max(contagem.items(), key=lambda x: x[1])[0]
                mensagens_finais.append(mensagem_mais_votada)
        return mensagens_finais

    def print_info(self):
        print("self.alpha: ",self.alpha)
        print("self.l: ",self.l)
        print("self.lambda: ",lambda_security)
        print("self.eps: ",self.eps)
        print("self.secrets: ",self.secrets)
        print("self.criterion: ",self.criterion)
        
    def print_secrets(self):
        print("[")
        for k in range(N):
            if k == self.b:
                print("None")
            else:
                print(self.secrets[k])
        print("]")

## Protocolo OLE

No protocolo OLE (“oblivious linear evaluation”) intervêm dois agentes, o Prover e o Verifier , que interagem da seguinte forma 

**Prover**

Este agente conhece um valor $\,x\in F\,$ , que é informação privada desse agente, e procede do seguinte modo:

1. Para todo $\,z \in E\,$ gera   $\,t_z \gets F\,$, se $z\neq 0\,$, e depois gera  $\,t_0\gets x - \sum_{z\neq 0}\,t_z\,$.
Note-se que se verifica $\,x\,=\,\sum_{z\in E}\,t_z\,$.
2. Disponibiliza $\,N = p^k\,$ mensagens $\,\{m_z \gets t_z\}_{z\in E}\,$ para transferência num protocolo “oblivious transfer”  $\,{N\choose{N-1}}$ .
3. Calcula o “tag”  $\;\mathsf{m}\,\gets\,\sum_{z\in E}\,(-z)\cdot t_z\,$  que  envia ao Verifier .

In [5]:
class Prover:
    def __init__(self):
        self.p = 3
        self.k = 5
        self.x = F(int.from_bytes(b"segredoX", byteorder='big'))
        self.sender = Sender(alpha="alpha123", l=10)
        self.t = None
        self.mensagens = None
        self.tag = 0
        self.E = GF(self.p^self.k, name='a', modulus=f)

    def init_sender(self):
        self.sender.get_criterion_sequence(lambda_security)
        
    #passo 1 do Prover
    def gerar_tz(self):
        t = {}
        soma = 0
        for z in self.E:
            if z != 0:
                t_z = F.random_element()
                t[z] = t_z
                soma += t_z
        t[0] = (self.x - soma)
        # Verificação
        soma_total = F(sum(t.values()))
        assert soma_total == self.x, f"Erro na soma: {soma_total} != {self.x}"
        self.t = t
        
    #passo 2 do Prover
    def disponibilizar_mensagens(self):
        self.mensagens = [self.t[z] for z in self.E]
        self.sender.mensagens = self.mensagens

    #passo 3 do Prover
    def calcula_tag(self):
        soma = self.E(0)
        for z in self.E:
            t_z = self.t[z]
            t_z_elem = self.E(t_z)
            soma += (-z) * t_z_elem
        self.tag = soma
        print(f"Tag calculado: {self.tag}")

In [6]:
prover = Prover()
prover.gerar_tz()
prover.init_sender()
prover.disponibilizar_mensagens()
prover.calcula_tag()

Tag calculado: a^4 + a^3 + a^2 + a


**Verifier**

1. Gera aleatoriamente $\,\Delta\,\gets\,E\,$ que passa a ser designada por chave global. 
2. No protocolo $\,{N}\choose{N-1}\,$-OT  inicializado pelo Prover, transfere as $\,N-1\,$ mensagens $\{m_z\}_{z\in E\setminus\Delta}$ 
3. Calcula  $\quad\mathsf{q}\,\gets\,\sum_{z\in E\setminus\Delta}\,(\Delta-z)\cdot m_z$

In [7]:
class Verifier:
    def __init__(self, eps, lambda_, n_mensagens):
        self.p = 3
        self.k = 5
        self.eps = eps
        self.lambda_ = lambda_
        self.n_mensagens = n_mensagens
        self.E = GF(self.p^self.k, name='a', modulus=f)
        self.delta = self.E.random_element() #passo 1 do Verifier
        self.receiver = Receiver(self.eps, list(self.E).index(self.delta), self.lambda_, self.n_mensagens)
        self.mensagens_escolhidas = None
        self.q = 0

    #passo 2 do Verifier
    def init_receiver(self):
        self.receiver.receive_alpha_l("alpha123", 10)
        self.receiver.generate_N_secrets()
        self.receiver.generate_pks()
    
    def recebe_mensagens(self,mensagens):
        self.mensagens_escolhidas = mensagens
    
    #passo 3 do Verifier
    def calcula_q(self):
        soma = self.E(0)
        elementos_E = sorted(list(self.E), key=str)  # Mesma ordem do Prover
        delta_index = elementos_E.index(self.delta)
        for i, z in enumerate(elementos_E):
            if z != self.delta:
                if self.mensagens_escolhidas[i] is not None:
                    # Converte bytes para inteiro e depois para F
                    t_z = F(int.from_bytes(self.mensagens_escolhidas[i], byteorder='big'))
                    soma += (self.delta - z) * self.E(t_z)
        self.q = soma

In [8]:
# Verifier
verifier = Verifier(0.1, 128, len(prover.sender.mensagens))
verifier.init_receiver()

# OT
prover.sender.receive_and_verifypks(verifier.receiver.t)
prover.sender.calcular_criptogramas()
print(len(prover.sender.criptogramas))

mensagens_decodificadas = verifier.receiver.recuperar_mensagens_maioritarias(prover.sender.criptogramas)

for i in range(0,len(prover.sender.mensagens)):
    if verifier.receiver.b != i:
        if prover.sender.mensagens[i] == mensagens_decodificadas[i]:
            print(f"Mensagem de índice {i} bem decifrada")
        else:
            print(f"Mensagem de índice {i} mal decifrada")
    else:
        if prover.sender.mensagens[i] != mensagens_decodificadas[i]:
            print(f"A mensagem de índice {i} que não era suposto decifrar foi corretamente mal decifrada")

mensagens_verifier = []
for msg in mensagens_decodificadas:
    if msg is not None:
        mensagens_verifier.append(msg)  # Já está no formato bytes (ex: b'1')
    else:
        mensagens_verifier.append(None)
verifier.recebe_mensagens(mensagens_verifier)

verifier.recebe_mensagens(mensagens_decodificadas)
verifier.calcula_q()

Verificação OK: todas as somas coincidem com u_i
243
Mensagem de índice 0 mal decifrada
Mensagem de índice 1 mal decifrada
Mensagem de índice 2 mal decifrada
Mensagem de índice 3 mal decifrada
Mensagem de índice 4 mal decifrada
Mensagem de índice 5 mal decifrada
Mensagem de índice 6 mal decifrada
Mensagem de índice 7 mal decifrada
Mensagem de índice 8 mal decifrada
Mensagem de índice 9 mal decifrada
Mensagem de índice 10 mal decifrada
Mensagem de índice 11 mal decifrada
Mensagem de índice 12 mal decifrada
Mensagem de índice 13 mal decifrada
Mensagem de índice 14 mal decifrada
Mensagem de índice 15 mal decifrada
Mensagem de índice 16 mal decifrada
Mensagem de índice 17 mal decifrada
Mensagem de índice 18 mal decifrada
Mensagem de índice 19 mal decifrada
Mensagem de índice 20 mal decifrada
Mensagem de índice 21 mal decifrada
Mensagem de índice 22 mal decifrada
Mensagem de índice 23 mal decifrada
Mensagem de índice 24 mal decifrada
Mensagem de índice 25 mal decifrada
Mensagem de índice 26

In [9]:
resultado = verifier.q == prover.x * verifier.delta + prover.tag
print(f"Verificação final: {resultado}")
print(f"q = {verifier.q}")
print(f"x*Δ + m = {prover.x * verifier.delta + prover.tag}")

Verificação final: False
q = 2*a^2 + a + 1
x*Δ + m = 2*a^3 + 2*a + 1


## Protocolo sVOLE

O protocolo OLE  simples tem simultaneamente uma questão de segurança e uma questão de eficiência ambas com a mesma proveniência: o número de elementos $\,p^k\,$ do corpo $E$.

- A questão de segurança resulta de, para existir capacidade de verificar o conhecimento de $\,x\in E\,$ ambos os “tags”  $\,\mathsf{m}\,,\,\mathsf{q}\in E\,$ têm de ser suficientemente grandes. Assim o tamanho dessas “tags”, que no protocolo OLE  é  $\,|p|\times k\,$, exige um valor mínimo razoável ($\sim 128\,$bits, ou superior).
- Ainda, em termos de segurança, o tamanho da chave global $\,\Delta\,$ tem de ser sufientemente grande para proteger o protocolo de ataques. Como $\,\Delta\,$ é um valor gerado de $\,E\,$ o factor “segurança de $\Delta$”  exige também um valor mínimo para $\,p^k\,$.
- A questão de eficiência resulta do facto de o número $\,N\,$ de mensagens usadas no protocolo OT  e usadas nas somas que calculam os “tags” ser também $\,p^k\,$. A eficiência exige que $\,N\,$ não possa ser muito grande.


Para criar uma solução que responda  simultaneamente a todas estas questões, a partir de um factor de segurança $\,\lambda\,$ e vamos seguir os seguintes objectivos

1. Os “tags” $\,\mathsf{m}\,$e $\,\mathsf{q}\,$ passam a ser vetores com $\,\ell\,$ componentes em $\,E\,$; como  elementos de $\,E^\ell\,$ o  o seu tamanho é substancialmente aumentado para  $\,|p|\times k\times \ell\,$;  se a dimensão $\,\ell\,$ for suficientemente grande, mesmo com $|p|$ e $\,k\,$ relativamente pequenos 
2. Os índices $\,z\,$ que identificam os elementos $\,t_z\,$ usados no protocolo OT   formam um subconjunto  $\,\mathcal{Z}\,\equiv\,\{z_1,z_2,\cdots\,z_N\}\,\subset E\,$ com $\,N  = \mathsf{poly}(\lambda)$  elementos.  
    O valor de $\,N\,$ e os elementos $\,\{z_n\}_{n\in[N]}\,$ são escolhidos de tal forma que, para todo $\,n\in [N]\,$ o conjunto $\,\mathcal{Z}\setminus\{z_n\}\,$ é uma base de $\,\mathbb{F}_{p^k}\,$ visto como espaço vectorial sobre $\,\mathbb{F}_p$.  
    Concretamente, cada $\,a\in E\,$ é uma combinação linear, com coeficientes em $\,F\,,$ dos elementos de $\,\mathcal{Z}\,$ e , cada $\,z_n\in\mathcal{Z}\,$ é uma combinação linear dos elementos de $\,\mathcal{Z}\setminus\{z_n\}\,$.
3. Os valores a autenticar $\,\overline{x}\,=\,\langle x_1, x_2,\cdots,x_\ell\rangle\,$ passam a ser também vectores de domensão $\,\ell\,$; isto é $\,\overline{x}\in F^{\ell}$.
4. A chave global $\,\Delta\,$ é escolhida, em cada instância do protocolo, dentro do  conjunto $\,\mathcal{Z}\,$ através da seleção de um índice $\,j\gets [N]\,$, e depois escolhendo $\,\Delta \equiv z_j\,$.
                                              $\Delta\,\gets\,\{\,\vartheta\,j\gets [N]\,\centerdot\, z_j\,\}$
5. Os valores $\,t_z\in F\,\,$ do protocolo OLE são substituídos por vectores; assim, para todo $\,i\in[\ell]\,$,  define-se um vector  $\,\overline{t}_i = \{t_{i,n}\}_{n\in[N]}\in\,F^\ell\,$ em que, 
        1. para $\,n>1\,$ , tem-se $\,t_{i,n}\,\gets\,F\,$ é gerado pseudo-aleatoriamente
        2. para  $\,n=1\,$  tem-se  $t_{i,1}\,\gets\, x_i - \sum_{n=2}^N\,t_{i,n}$.
    Isto assegura que, para todo $\,i\in[\ell]\,$, se verifica $\,x_i \,=\,\sum_{n\in[N]}\,t_{i,n}$



6. Ambos os agentes, Prover e Verifier  conhecem os parâmetros  $\,p,k,\ell\,$ e o parâmetro de segurança $\,\lambda\,$. Conhecem também $\,\mathcal{Z} \subset \mathbb{F}_{p^k}\,$ e a sua cardinalidade $\,N=O(\mathsf{poly}(\lambda))\,$, como especificámos atrás.

### Parâmetros globais

In [10]:
p = 3 
k = 5  
l = 8 
lambda_security = 128 
N = 2 * k 

### Estruturas algébricas

In [11]:
F = GF(p)
R.<x> = PolynomialRing(F)
f = R.irreducible_element(k)
E.<a> = GF(p^k, name='a', modulus=f)  

### Prover sVOLE

Este agente conhece um vector $\,\overline{x}\in F^\ell\,$ , que é sua informação privada, e procede do seguinte modo 

a. para todo $\,i\in [\ell]$ \
   i. Gera o vector $\,\overline{t}_i\,$  como está indicado no ponto 5. acima, \
   ii. Disponibiliza $\,N\,$ mensagens $\,\{m_{i,n} \gets \sigma(t_{i,n})\}_{n\in [N]}\,$   para transferência num protocolo “oblivious transfer”  $\,{N\choose{N-1}}$ . \
   iii. Calcula o “tag”  $\;\mathsf{m}_i\,\gets\,\sum_{n\in[N]}\,(-z_n)\cdot t_{i,n}\,$.

b. Agrega num vector $\;\overline{\mathsf{m}}\in E^\ell\;$ os vários “tags”  $\,\{\mathsf{m}_i\}_{i\in [\ell]}$

In [12]:
class Prover_sVOLE:
    def __init__(self):
        self.vectorX = vector(F, [F.random_element() for _ in range(l)])
        self.t_matrix = {}
        self.messages = []

    #passo a.i
    def gerar_tz(self):
        for i in range(l):  
            t_i = {n: F.random_element() for n in range(2, N + 1)}
            
            t_i[1] = self.vectorX[i] - sum(t_i[n] for n in range(2, N + 1))
            
            # debug
            assert self.vectorX[i] == sum(t_i[n] for n in range(1, N + 1)), \
                   f"Erro na soma para x[{i}]"

            #sort simples
            t_keys = list(t_i.keys())
            t_keys.sort()
            
            self.t_matrix[i] = {i:t_i[i] for i in t_keys}

    def mostrar_estrutura(self):
        print(f"Vetor privado x: {self.vectorX}")
        print("\nMatriz t_{i,n}:")
        for i in range(min(3, l)):  # Mostra apenas as 3 primeiras linhas para exemplo
            print(f"i={i}: {self.t_matrix.get(i, 'Não gerado')}")

    #ponto a.ii
    def gen_messages(self):
        self.messages = {} 
        for i in range(l): self.messages[i] = {n: self._sigma(self.t_matrix[i][n]) for n in range(1, N + 1)}
        print("self.messages: ",self.messages)

    # Função sigma que codigica um elemento de F como string de bits
    def _sigma(self, x):
        bit_length = (p - 1).bit_length()
        
        if x in F: 
            return format(int(x), f'0{bit_length}b')
        elif isinstance(x, list) and all(y in F for y in x):
            return ''.join([format(int(y), f'0{bit_length}b') for y in x])
        else:
            raise ValueError("x deve ser um elemento de F ou uma lista de elementos de F")

    #ponto a.iii
            
prover_sVOLE = Prover_sVOLE()
prover_sVOLE.gerar_tz()
prover_sVOLE.mostrar_estrutura()
prover_sVOLE.gen_messages()

Vetor privado x: (0, 0, 1, 2, 0, 2, 1, 1)

Matriz t_{i,n}:
i=0: {1: 0, 2: 1, 3: 1, 4: 0, 5: 1, 6: 0, 7: 2, 8: 1, 9: 2, 10: 1}
i=1: {1: 2, 2: 1, 3: 0, 4: 0, 5: 0, 6: 1, 7: 2, 8: 0, 9: 2, 10: 1}
i=2: {1: 0, 2: 1, 3: 2, 4: 2, 5: 2, 6: 1, 7: 0, 8: 2, 9: 2, 10: 1}
self.messages:  {0: {1: '00', 2: '01', 3: '01', 4: '00', 5: '01', 6: '00', 7: '10', 8: '01', 9: '10', 10: '01'}, 1: {1: '10', 2: '01', 3: '00', 4: '00', 5: '00', 6: '01', 7: '10', 8: '00', 9: '10', 10: '01'}, 2: {1: '00', 2: '01', 3: '10', 4: '10', 5: '10', 6: '01', 7: '00', 8: '10', 9: '10', 10: '01'}, 3: {1: '00', 2: '10', 3: '00', 4: '01', 5: '00', 6: '01', 7: '10', 8: '01', 9: '01', 10: '00'}, 4: {1: '10', 2: '10', 3: '10', 4: '00', 5: '00', 6: '00', 7: '01', 8: '00', 9: '01', 10: '01'}, 5: {1: '10', 2: '10', 3: '00', 4: '10', 5: '01', 6: '10', 7: '10', 8: '10', 9: '01', 10: '00'}, 6: {1: '00', 2: '01', 3: '00', 4: '01', 5: '10', 6: '00', 7: '10', 8: '10', 9: '10', 10: '00'}, 7: {1: '01', 2: '10', 3: '10', 4: '10', 5: '00', 6: